# Serverless Platform System Measurements for Simulator

In [1]:
import asyncio
from sf_platform import ServerlessPlatform
import numpy as np
import pandas as pd

In [2]:
sp = ServerlessPlatform("./sf_platform/template/")

In [3]:
for i in range(10):
    await sp.register_function(f"sleep-{i}", "./sf_platform/examples/sleep/entry.py", "./sf_platform/examples/sleep/requirements.txt")
    
    # Cold starts
    await asyncio.gather(*[sp.run_function(f"sleep-{i}", query_params="t=0") for _ in range(10)])
    
    await asyncio.sleep(1)
    
    # Warm starts
    await asyncio.gather(*[sp.run_function(f"sleep-{i}", query_params="t=0") for _ in range(10)])
    
    await asyncio.sleep(1)

pass # Supress notebook output

In [4]:
# "acquire" = time from entry to getting a container (when warm, directly get one from available, otherwise make one)
warm_acquire_durations = []
cold_acquire_durations = []

# "execution_overhead" = time from getting the result from getting the container (represents routing the request across 
# the datacenter and waiting for the machines to marshall/unmarshall)
# assumes 0 execution time for the actual program! (we would slot the execution time of whatever function we're trying to model
# after this duration)
warm_execution_overhead_durations = []
cold_execution_overhead_durations = []

# "release" = time from getting a response to the time the container is ready for the next execution
warm_release_durations = []
cold_release_durations = []

for row in sp.logs:
    if row['log_type'] != 'invocation_trace':
        continue
        
    acquire = row['container_acquire_time'] - row['entry_time']
    execution_overhead = row['response_time'] - row['container_acquire_time']
    release = row['container_release_time'] - row['response_time']
    
    if row['cold_start']:
        cold_acquire_durations.append(acquire)
        cold_execution_overhead_durations.append(execution_overhead)
        cold_release_durations.append(release)
    else:
        warm_acquire_durations.append(acquire)
        warm_execution_overhead_durations.append(execution_overhead)
        warm_release_durations.append(release)


### Warm Starts

In [6]:
warm_df = pd.DataFrame({
    "Acquire": warm_acquire_durations,
    "Execution Overhead": warm_execution_overhead_durations,
    "Release": warm_release_durations
})


warm_df.describe()

,Acquire,Execution Overhead,Release
count,100.000000,100.000000,100.000000
mean,0.000017,0.021617,0.000007
std,0.000015,0.010093,0.000005
min,0.000003,0.008964,0.000003
25%,0.000009,0.014080,0.000005
50%,0.000011,0.017489,0.000006
75%,0.000022,0.029753,0.000008
max,0.000111,0.044499,0.000040


### Cold Starts

In [7]:
cold_df = pd.DataFrame({
    "Acquire": cold_acquire_durations,
    "Execution Overhead": cold_execution_overhead_durations,
    "Release": cold_release_durations
})


cold_df.describe()

,Acquire,Execution Overhead,Release
count,100.000000,100.000000,100.000000
mean,1.121333,0.220730,0.000013
std,0.629269,0.045443,0.000009
min,0.140338,0.113523,0.000005
25%,0.605085,0.220165,0.000007
50%,1.083161,0.226644,0.000014
75%,1.627411,0.236013,0.000015
max,2.368496,0.349994,0.000075


### Combined Statistics

In [8]:
together_df = pd.DataFrame({
    "Acquire": cold_acquire_durations + warm_acquire_durations,
    "Execution Overhead": cold_execution_overhead_durations + warm_execution_overhead_durations,
    "Release": cold_release_durations + warm_release_durations
})


together_df.describe()

,Acquire,Execution Overhead,Release
count,200.000000,200.000000,200.000000
mean,0.560675,0.121173,0.000010
std,0.716179,0.105068,0.000008
min,0.000003,0.008964,0.000003
25%,0.000011,0.017545,0.000006
50%,0.070225,0.079011,0.000007
75%,1.074419,0.226620,0.000014
max,2.368496,0.349994,0.000075


In [9]:
sp.shutdown()